In [4]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException

firefox_options = FirefoxOptions()
firefox_options.add_argument("-headless")
firefox_options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"

# Get the path to the GeckoDriver executable
gecko_driver_path = r"C:\Users\ahmed\Downloads\geckodriver-v0.33.0-win64\geckodriver.exe"
service = Service(executable_path=gecko_driver_path)

# Initialize the driver using the GeckoDriver executable and Firefox options
driver = webdriver.Firefox(service=service, options=firefox_options)

driver.get("https://globalink.mitacs.ca/#/student/application/projects")

def nextPage():#p-icon p-paginator-icon
    elements = driver.find_elements(By.XPATH,'//button[@class="p-ripple p-element p-paginator-next p-paginator-element p-link"]')[0]
    print(elements)
    # elements.click()
    driver.execute_script("arguments[0].click();", elements)

def popup(element,i):
    elements = element.find_elements(By.XPATH,"//span[@class='p-button-label' and text()='View Detail']")[i]
    # elements.click()
    driver.execute_script("arguments[0].click();", elements)
def more():
    elements = driver.find_elements(By.XPATH, "//a[@id='p-tabpanel-2-label']")
    if elements:
        try:
            driver.execute_script("arguments[0].click();", elements[0])
        except StaleElementReferenceException:
            pass  # Ignore the exception and continue
    else:
        print("No 'More' element found, skipping action")

"""def more():
    elements = driver.find_elements(By.XPATH,"//a[@id='p-tabpanel-2-label']")[0]
    #wait = WebDriverWait(driver, 25)
    #elements = wait.until(EC.element_to_be_clickable((By.ID,'p-tabpanel-2-label')))
    #print(elements)
    # elements.click()
    driver.execute_script("arguments[0].click();", elements)"""
def close():
    driver.find_element(By.XPATH, "//body").send_keys(Keys.ESCAPE)

def execone():
    elements = driver.find_elements(By.XPATH,"//div[contains(@class, 'col-12') and contains(@class, 'col-md-12') and contains(@class, 'ng-star-inserted')]")
    curPID = 0
    for i, element in enumerate(elements, start=0):
        idName = element.find_elements(By.CLASS_NAME,'row')
        header = idName[0].text.split('\n')
        # name = header[-1]
        #name = name.replace(',',';')
        pid = header[0].split()[-1]
        curPID = pid
        description = idName[1].find_elements(By.CLASS_NAME,'col-9')[0].text

        name = description.split('\n')[0]
        name = name.replace(',',';')
        description = description.split('\n')[1].replace(',',';')
        hostDetails = element.find_element(By.CLASS_NAME,'col-3').text.split('\n')
        # print(profs[0].text)
        prof = hostDetails[0].split(':')[-1].strip()
        prof = prof.replace(',',';')
        hostProvince = hostDetails[1].split(':')[-1].strip()
        hostProvince = hostProvince.replace(',',';')
        hostUniv = hostDetails[2].split(':')[-1].strip()
        hostUniv = hostUniv.replace(',',';')
        language = hostDetails[5].split(':')[-1].strip()
        language = language.replace(',',';')
        startDate=  hostDetails[6].split(':')[-1].strip().split('(')[0].strip()
        startDate = startDate.replace(',',';')
        popup(element,i)
        hehe=driver.find_elements(By.CLASS_NAME,'detailRow')
        if (len(hehe)>=7):
            try:
                hehehe = hehe[6].find_elements(By.TAG_NAME, 'td')[-1].text
            except StaleElementReferenceException:
                hehehe = 'None'
        else:
            hehehe='None'
        hehehe=hehehe.replace(',',';')
        more()
        #new_driver=driver.find_element(By.CLASS_NAME,'p-dialog-draggable')
        skills = driver.find_elements(By.ID, 'p-tabpanel-2')
        if len(skills) > 0:
            try:
                skill_element = skills[0].find_elements(By.TAG_NAME, 'p')[0]
                skill = skill_element.text
            except StaleElementReferenceException:
                pass
            # Proceed with your code here
        else:
            skill = "No skills available" 
        close()
        skill = skill.replace(',', ';')
        print(skill+ '\n')
        writeFile.write(pid.replace(',', ';') + ',' + name.replace(',', ';') +',' + description.replace(',', ';') + ',' + prof.replace(',', ';') + ',' + hostProvince.replace(',', ';') + ',' + hostUniv.replace(',', ';') + ',' + language.replace(',', ';') + ',' +startDate.replace(',', ';')+','+skill.replace(',', ';')+','+ str(hehehe).replace(',', ';') + '\n')
    print(s, len(elements), curPID)
    nextPage()
# ... (Previous code remains the same)
"""
def extract_data(element):
    try:
        id_name_element = element.find_element(By.CLASS_NAME, 'ui-card-title')
        pid = id_name_element.find_element(By.TAG_NAME, 'span').text
        name = id_name_element.find_element(By.TAG_NAME, 'h3').text

        description = element.find_element(By.CLASS_NAME, 'ui-card-content').text.replace(",", ";")

        host_details = element.find_element(By.CLASS_NAME, 'ui-card-footer').text.split('\n')
        prof = host_details[1].split(': ')[1].replace(",", ";")
        host_province = host_details[2].split(': ')[1].replace(",", ";")
        host_univ = host_details[3].split(': ')[1].replace(",", ";")
        language = host_details[4].split(': ')[1].replace(",", ";")
        start_date = host_details[5].split(': ')[1].split(' (')[0].replace(",", ";")
        
        return f"{pid},{name},{description},{prof},{host_province},{host_univ},{language},{start_date}\n"
    except Exception as e:
        print("An error occurred while extracting data:", str(e))
        return ""
"""
# Start from the first page
writeFile = open('output_test.csv', 'w', encoding="utf-8")

# Header
writeFile.write('Project ID' + ',' + 'Name' +',' + 'Description' + ',' + 'Faculty supervisor' + ',' + 'Host Province' + ',' + 'Host Institution' + ',' + 'Language' + ',' + 'Preferred start date'+','+'Skills'+','+'Student academic background' + '\n')

s = 0
while s <=296:
    s += 1
    # elements = driver.find_elements_by_xpath("//div[contains(@class, 'ui-blockui') and contains(@class, 'ui-widget-overlay')]")
    # print(len(elements))
    a = 1
    while a > 0:
        a = len(driver.find_elements(By.ID,'waitBox'))
        time.sleep(0.1)
        print('.',end = '')
    print()
    execone()
writeFile.close()

print(f"Data extraction complete for {s} elements.")

.
The ideal will student will have strong background in computational biology; biochemistry and machine learning. The undergraduate program can be in computer science of engineering and the student will have experience with bioinformatics and genome sequence analysis. The student must be willing to learning techniques and work with senior researchers in order to execute new ideas and algorithms into code and test them out. Finally; the student should be comfortable programming in Python and familiarity with Pytorch and developing code for GPUs.

No 'More' element found, skipping action
No skills available

1. Background in tumor biology and immunology (e.g. courses or summer project) 2. Cell culture 3. Cell based assays. e.g. MTT/XTT assay and ELISA 4. English proficiency 5. Microsoft Office (Powerpoint; Excel; Word)

1. Background in drug delivery; pharmaceutics or chemical engineering (e.g. courses or summer project) 2. Cell culture 3. Molecular biology assays; such as qPCR; Western 

IndexError: list index out of range